In [1]:
import numpy as np
import pandas as pd
import os
import gc


import json
import math 
import sys
from datetime import datetime
import time
from typing import Tuple,List

import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import sklearn.metrics
from PIL import Image

from matplotlib.axes import Axes
from matplotlib import animation, rc
import plotly.graph_objs as go
import plotly.tools as ts
from plotly.offline import plot, init_notebook_mode
import plotly.figure_factory as ft
init_notebook_mode(connected=True)
from pyquaternion import Quaternion
import seaborn as sns
import warnings

from lyft_dataset_sdk.utils.map_mask import MapMask
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.geometry_utils import view_points, box_in_image, BoxVisibility
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from pathlib import Path

from lyft_dataset_sdk.lyftdataset import LyftDataset,LyftDatasetExplorer
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
import time
from lyft_dataset_sdk.utils.map_mask import MapMask

import struct
from abc import ABC, abstractmethod
from functools import reduce
from typing import Tuple, List, Dict
import copy
from shutil import copyfile

from PIL import Image

In [2]:
test_dataset = LyftDataset(data_path='.', json_path='../../3d-object-detection-for-autonomous-vehicles/test_data', verbose=True)

JSON file instance.json missing, using empty list
JSON file sample_annotation.json missing, using empty list
9 category,
17 attribute,
4 visibility,
0 instance,
8 sensor,
168 calibrated_sensor,
219744 ego_pose,
218 log,
218 scene,
27468 sample,
219744 sample_data,
0 sample_annotation,
1 map,
Done loading in 2.0 seconds.
Reverse indexing ...
Done reverse indexing in 0.6 seconds.


In [3]:
def lidar_filepath_from_id(sample_token):
    sample = test_dataset.get("sample", sample_token)

    sample_lidar_token = sample["data"]["LIDAR_TOP"]
    
    lidar_path = test_dataset.get_sample_data_path(sample_lidar_token)
    return lidar_path
def write_image_from_id(image_folder,kitti_id,sample_token):
    

    sample = test_dataset.get("sample", sample_token)

    sample_cam_token = sample["data"]["CAM_FRONT_LEFT"]
    cam_data = test_dataset.get("sample_data", sample_cam_token)
    cam_filepath = test_dataset.get_sample_data_path(sample_cam_token)
    
    im = Image.open(cam_filepath)
    im.save(os.path.join(image_folder,"{:06n}".format(kitti_id)+".png"), "PNG")
def write_calib(calib_folder,idx):
    
    label_save_fn = os.path.join(calib_folder,"{:06n}".format(idx)+'.txt')
    Tr_lyft_rect = np.array([-1,  0,  0,  0, 0,  0, -1,  0, 0,  1,  0,  0])
    R0_rect = np.array([1,0,0,0,1,0,0,0,1])
    with open(label_save_fn, 'w') as f:

        print('P0: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)
        print('P1: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)
        print('P2: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)
        print('P3: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f'  %
                          (1,0,0,0,0,1,0,0,0,0,1,0),file=f)

        print('R0_rect: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f' %
                          (R0_rect[0], R0_rect[1],R0_rect[2], R0_rect[3], R0_rect[4],
                           R0_rect[5], R0_rect[6], R0_rect[7], R0_rect[8]),file=f)

        print('Tr_velo_to_cam: %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f %.4f' %
                          (-1,0,0,0,0,0,-1,0,0,1,0,0),file=f)

In [4]:

calib_folder = "/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/calib/"
lidar_folder = '/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/velodyne'
image_folder = '/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/image_2'
testtxtfile = '/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/ImageSets/test.txt'

os.makedirs(calib_folder, exist_ok=True)
os.makedirs(lidar_folder, exist_ok=True)
os.makedirs(image_folder, exist_ok=True)


#os.makedirs(lidar_folder)

df = pd.read_csv('sample_submission.csv')
lyfttest_Id = df['Id']

In [11]:
lidar_folder_1 = '/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/velodyne_1'
for kitti_id in range(100):
    sample_token = lyfttest_Id[kitti_id]
    """
        prepare test.txt, calibration file and the lidar files
    """
    # write test.txt
    if kitti_id % 10 == 0:
        with open(testtxtfile, 'w') as f:
            print("{:06n}".format(kitti_id),file=f)
    else:
        with open(testtxtfile, 'a') as f:
            print("{:06n}".format(kitti_id),file=f)
    
    # write calibration file
    write_calib(calib_folder,kitti_id)
    
    # extract lidar file path
    lidar_filepath = lidar_filepath_from_id(sample_token)
    # move lidar to path
    lidar_savefilename = "{:06n}".format(kitti_id)+'.bin'
    copyfile(lidar_filepath,os.path.join(lidar_folder_1, lidar_savefilename)) 
    # write image
    write_image_from_id(image_folder,kitti_id,sample_token)

In [9]:
lidar_folder

'/home/lingling/Downloads/Lyft_PointRCNN/data/KITTI/object/testing/velodyne'

In [10]:
!mkdir 